In [141]:
import numpy as np
#import autodiff

class autodiff():
    def __init__(self,name,val,der=1):
        self.name = name
        self.val = val
        self.der = {name:der}
        #self.lparent = None
        #self.rparent = None
        # self.child = None
        #self.back_der = {name:None}
        self.funcder = [1]
        self.back_der = 1
        
    def backprop(self):
#         print(self.lparent, self.rparent)
#         try:
#             print(self.rparent.val)
#             self.back_der['left'] = self.rparent.val*val
#         except:
#             self.back_der['left'] = self.rparent*val
#         try:
#             print(self.lparent.val)
#             self.back_der['right'] = self.lparent.val*val
#         except:
#             self.back_der['right'] = self.lparent*val
# #             for key in self.der:
# #                 self.back_der[key] = self.der[key]*val #self.val
# #                 print(self.back_der[key])
# #                 self.lparent = self.lparent.lparent
    
        self.back_der = self.back_der* self.funcder
        #need to iter and backtrack
                
        return self.back_der
        
    
    def __mul__(self, other):
        """Allows multiplication of another autodiff instance, or multiplication of a constant (integer or float)."""

        if isinstance(other, (int, float, autodiff)) == False:
            raise ValueError("Error: Only integer, float, or autodiff instances can be multiplied.")

        anew = autodiff(self.name, self.val, self.der)
        
        #Stores for backpropagation functionality
        #anew.lparent = self
        #anew.rparent = other
        #self.child = anew
        #other.child = anew
        try:
            #print(self.funcder)
            anew.funcder = self.funcder
            anew.funcder.append(other.funcder)  #when other is autodiff object
        except:
            anew.funcder =self.funcder
            anew.funcder.append(other) #when other is a value

        #assuming that other is autodiff instance
        try:
            anew.val = self.val*other.val
            for key in np.unique([key for key in self.der] + [key for key in other.der]):
                if key not in self.der:
                    anew.der[key]=self.val*other.der[key]
                elif key not in other.der:
                    anew.der[key]=self.der[key]*other.val
                else:
                    anew.der[key]=self.der[key]*other.val+other.der[key]*self.val
            
        # if 'other' is not autodiff instance
        except AttributeError:
            # assuming that 'other' is a valid constant
            for key in self.der:
                anew.der[key] = other*self.der[key]
                anew.val = other*self.val

        
        return anew

    __rmul__ = __mul__
    
    
    def __neg__(self):
        """Allows unary operation of autodiff instance."""
        anew = autodiff(self.name, -self.val, self.der)
        for key in self.der:
            anew.der[key] = -self.der[key]
        anew.funcder = self.funcder
        anew.funcder.append(-1)
        return anew


    def __truediv__(self,other):
        '''function for left division'''

        if isinstance(other, (int, float, autodiff)) == False:
            raise ValueError("Error: Only integer, float, or autodiff instances can be divided.")

        anew = autodiff(self.name, self.val, self.der)
        try:
            anew.val = self.val/other.val
            for key in np.unique([key for key in self.der] + [key for key in other.der]):
                if key not in self.der:
                    anew.der[key]=self.val*other.der[key]/other.val**2
                elif key not in other.der:
                    anew.der[key]=self.der[key]/other.val
                else:
                    anew.der[key]=0
        except AttributeError:
            for key in self.der:
                anew.der[key] = self.der[key]/other
                anew.val = self.val/other
        try:
            anew.funcder = self.funcder
            anew.funcder.append(self.val/other.val)
        except:
            anew.funcder = self.funcder
            anew.funcder.append(self.val/other)
            
            
        return anew


    def __rtruediv__(self, other):

        '''function for right division, when performing (constant)/(autodiff)'''

        if isinstance(other, (int,float)):
            anew = autodiff(self.name, self.val, self.der)
            for key in self.der:
                anew.der[key] = -other*self.der[key]/self.val**2
                anew.val = other/self.val
            
            try:
                anew.funcder = self.funcder
                anew.funcder.append(-other.val/(self.val**2))
            except:
                anew.funcder = self.funcder
                anew.funcder.append(-other/(self.val**2))
            
            return anew
        else:
            raise ValueError("Error: Only integer, float, or autodiff instances can be divided.")



    def __add__(self, other):

        if isinstance(other, (int, float, autodiff)) == False:
            raise ValueError("Error: Only integer, float, or autodiff instances can be added.")


        #Generate a new autodiff instance copy of self
        anew = autodiff(self.name, self.val, self.der)
        #Tries adding two autodiff instances together
        anew.funcder = self.funcder
        anew.funcder.append(1)
        #anew.lparent = self
        #anew.rparent = other
        
        try:

            #Add values
            anew.val = self.val + other.val

            #Calculate derivatives of this addition for all variables so far encountered
            for key in np.unique([key for key in self.der] + [key for key in other.der]): #Iterate through all unique variables so far encountered
                #If self has not encountered this variable before (so derivative of self with respect to variable must be 0)
                if key not in self.der:
                    anew.der[key] = other.der[key]

                #Else, if opponent has not encountered this variable before (so derivative of self with respect to variable must be 0)
                elif key not in other.der:
                    anew.der[key] = self.der[key]

                #Else, if both self and opponent have encountered this variable before
                else:
                    anew.der[key] = self.der[key] + other.der[key]

        #Otherwise, if not two autodiff instances:
        except AttributeError:
            #Tries adding autodiff instance and number together

            for key in self.der:
                anew.der[key] = self.der[key]
                anew.val = other + self.val

        #Returns new autodiff instance
        return anew



    #FUNCTION: __radd__
    #PURPOSE: Allows commutative addition.
    def __radd__(self, other):
        return self + other

    #FUNCTION: __sub__
    #PURPOSE: Subtract an autodiff instance or number from a autodiff instance, and calculate the derivatives resulting from this action.
    def __sub__(self, other):
        
        if isinstance(other, (int, float, autodiff)) == False:
            raise ValueError("Error: Only integer, float, or autodiff instances can be subtracted.")

        #Generate a new autodiff instance copy of self
        anew = autodiff(self.name, self.val, self.der)
        #anew.lparent = self
        #anew.rparent = other
        #Tries subtracting two autodiff instances together
        try:
            #Subtract values
            anew.val = self.val - other.val

            #Calculate derivatives of this subtraction for all variables so far encountered
            for key in np.unique([key for key in self.der] + [key for key in other.der]): #Iterate through all unique variables so far encountered
                #If self has not encountered this variable before (so derivative of self with respect to variable must be 0)
                if key not in self.der:
                    anew.der[key] = -1*other.der[key]

                #Else, if opponent has not encountered this variable before (so derivative of self with respect to variable must be 0)
                elif key not in other.der:
                    anew.der[key] = self.der[key]

                #Else, if both self and opponent have encountered this variable before
                else:
                    anew.der[key] = self.der[key] - other.der[key]

        #Otherwise, if not two autodiff instances:
        except AttributeError:
            #Tries subtracting number from autodiff instance

            for key in self.der:
                anew.der[key] = self.der[key]
                anew.val = self.val - other
        anew.funcder = self.funcder
        anew.funcder.append(1)
        #Returns new autodiff instance
        return anew



    #FUNCTION: __sub__
    #PURPOSE: Subtract an autodiff instance or number from a autodiff instance, and calculate the derivatives resulting from this action.
    def __rsub__(self, other):
        return (-1*self) + other

      
def autoexp(ad):
    '''Returns autodiff instance of exp(x)

    INPUTS
    ==========
    ad: autodiff instance

    RETURNS
    ==========
    anew: autodiff instance with updated values and derivatives

    EXAMPLES
    ==========
    >>> from autodiffpy import autodiff
    >>> from autodiffpy import autodiff_math as admath
    >>> x = autodiff.autodiff('x', 10)
    >>> f1 = admath.exp(x)
    >>> f1.val = np.exp(10)
    '''

    try:
        
        anew = autodiff(name=ad.name, val = np.exp(ad.val), der = ad.der)
        for key in ad.der:
            anew.der[key] = ad.der[key]*anew.val

        return anew
    except AttributeError:
        raise AttributeError("Error: input should be autodiff instance only.")

In [148]:
#same case in the slide
x1 = autodiff(name="x1", val=3, der=1)
w1 = 1.0
x2 = autodiff(name="x2", val=2, der=1)
w2 = -2.0
w0 = 2.0
f1 = w1*x1

print(f1.funcder)

[1, 1.0]


In [149]:
f2= w2*x2
print(f2.funcder)
#print(f2.backprop())
#print(f1.val, f2.val)
f1 = f1+f2
print(f1.backprop())
#print(f1.der, f1.lparent.der,  f1.rparent.der, f1.val)
f1 = w0+f1
print(f1.backprop())
#print(f1.der, f1.lparent.der, f1.val)
f1= 0-f1
#print(f1.der, f1.lparent.der, f1.rparent, f1.val)
f = 1/(1+autoexp(f1))
print(f.val)
print(f.der)
print(f.funcder)

[1, -2.0]
[1, 1.0, 1]
[1, 1.0, 1, 1]
0.7310585786300049
{'x1': 0.19661193324148188, 'x2': -0.39322386648296376}
[1, 1, -0.534446645388523]


In [147]:
print(f.backprop())

[1, 1, -0.002249213446654648]


In [151]:
#cory's
import numpy as np


class autodiff():
    def __init__(self,name,val,der=1):
        self.name = name
        self.val = val
        self.der = {name:der}
        self.lparent = None
        self.rparent = None
        # self.child = None
        self.back_der = None
        self.back_partial_der = None
        self.func = None
        
    def backprop(self, backproplist = None):
        if backproplist == None:
            backproplist = []
            self.back_der = 0.73 #1 #0.73
        if self.lparent:
            try:
                self.lparent.back_der = self.back_der*self.lparent.back_partial_der
                self.lparent.backprop(backproplist)
            except:
                pass
        if self.rparent:
            try:
                self.rparent.back_der = self.back_der*self.rparent.back_partial_der
                self.rparent.backprop(backproplist)
            except:
                pass
        if self.lparent is None and self.rparent is None:
            backproplist.append((self.name,self.back_partial_der,self.back_der))
        
        return backproplist
        
    
    def __mul__(self, other):
        """Allows multiplication of another autodiff instance, or multiplication of a constant (integer or float)."""

        if isinstance(other, (int, float, autodiff)) == False:
            raise ValueError("Error: Only integer, float, or autodiff instances can be multiplied.")

        anew = autodiff(self.name, self.val, self.der)
        
        #Stores for backpropagation functionality
        anew.lparent = self
        anew.rparent = other
        #self.child = anew
        #other.child = anew
        anew.func = self.__mul__

        #assuming that other is autodiff instance
        try:
            anew.val = self.val*other.val
            for key in np.unique([key for key in self.der] + [key for key in other.der]):
                if key not in self.der:
                    anew.der[key]=self.val*other.der[key]
                elif key not in other.der:
                    anew.der[key]=self.der[key]*other.val
                else:
                    anew.der[key]=self.der[key]*other.val+other.der[key]*self.val
            #set the back partial derivatives that can be used for backpropagation

            self.back_partial_der = other.val
            other.back_partial_der = self.val
            
        # if 'other' is not autodiff instance
        except AttributeError:
            # assuming that 'other' is a valid constant
            for key in self.der:
                anew.der[key] = other*self.der[key]
                anew.val = other*self.val
                self.back_partial_der = other

        return anew

    __rmul__ = __mul__


    def __add__(self, other):

        if isinstance(other, str):
            raise ValueError("Error: input cannot be string type.")

        #Generate a new autodiff instance copy of self
        anew = autodiff(self.name, self.val, self.der)

        anew.lparent = self
        anew.rparent = other

        #Tries adding two autodiff instances together
        try:

            #Add values
            anew.val = self.val + other.val

            #Calculate derivatives of this addition for all variables so far encountered
            for key in np.unique([key for key in self.der] + [key for key in other.der]): #Iterate through all unique variables so far encountered
                #If self has not encountered this variable before (so derivative of self with respect to variable must be 0)
                if key not in self.der:
                    anew.der[key] = other.der[key]

                #Else, if opponent has not encountered this variable before (so derivative of self with respect to variable must be 0)
                elif key not in other.der:
                    anew.der[key] = self.der[key]

                #Else, if both self and opponent have encountered this variable before
                else:
                    anew.der[key] = self.der[key] + other.der[key]

            self.back_partial_der = 1
            other.back_partial_der = 1

        #Otherwise, if not two autodiff instances:
        except AttributeError:
            #Tries adding autodiff instance and number together

            try:
                for key in self.der:
                    anew.der[key] = self.der[key]
                    anew.val = other + self.val
                self.back_partial_der = 1

            #Otherwise, raises a type error if not compatible
            except:
                raise TypeError('Error: please input a number or autodiff class')

        #Returns new autodiff instance
        return anew

    __radd__=__add__

    def __pow__(self, other):

        if isinstance(other, str):
            raise ValueError("Error: input cannot be string type.")

        #Generate a new autodiff instance copy of self
        anew = autodiff(self.name, self.val, self.der)

        anew.lparent = self
        anew.rparent = other

        #Tries raising this autodiff instance to another autodiff instance
        try:
            #Raise values
            anew.val = self.val**other.val

            #Calculate derivatives of this exponentiation for all variables so far encountered
            for key in np.unique([key for key in self.der] + [key for key in other.der]): #Iterate through all unique variables so far encountered
                #If self has not encountered this variable before (so derivative of self with respect to variable must be 0)
                if key not in self.der:
                    anew.der[key] = anew.val*(np.log(self.val)*other.der[key])

                #Else, if opponent has not encountered this variable before (so derivative of self with respect to variable must be 0)
                elif key not in other.der:
                    anew.der[key] = anew.val*(other.val*self.der[key]/1.0/self.val)

                #Else, if both self and opponent have encountered this variable before
                else:
                    anew.der[key] = anew.val*((np.log(self.val)*other.der[key]) + (other.val*self.der[key]/1.0/self.val))

            self.back_partial_der = other.val*self.val**(other.val-1)
            other.back_partial_der = (self.val**other.val)*np.log(self.val)

        #Otherwise, if not two autodiff instances:
        except AttributeError:
            #Tries adding autodiff instance and number together

            try:
                for key in self.der:
                    anew.der[key] = other*(self.val**(other - 1))*self.der[key]
                    anew.val = self.val**other
                self.back_partial_der = other*self.val**(other-1)
            #Otherwise, raises a type error if not compatible
            except:
                raise TypeError('Error: please input a number or autodiff class')

        #Returns new autodiff instance
        return anew




def exp(ad):
    '''Returns autodiff instance of log(x)

    INPUTS
    ==========
    ad: autodiff instance

    RETURNS
    ==========
    anew: autodiff instance with updated values and derivatives

    EXAMPLES
    ==========
    >>> from autodiffpy import autodiff
    >>> from autodiffpy import autodiff_math as admath
    >>> x = autodiff.autodiff('x', 10)
    >>> f1 = admath.exp(x)
    >>> f1.val = np.exp(10)
    '''
    

    try:
        anew = autodiff(name=ad.name, val = np.exp(ad.val), der = ad.der)
        anew.lparent = ad
        for key in ad.der:
            anew.der[key] = ad.der[key]*anew.val
        ad.back_partial_der = anew.val
        return anew
    except TypeError:
        print("Error: input should be autodiff instance only.")




x1 = autodiff(name="x1", val=3, der=1)
x2 = autodiff(name="x2", val=2, der=1)
w0 = autodiff(name="w0", val=2, der=1)
w1 = autodiff(name="w1", val=1, der=1)
w2 = autodiff(name="w2", val=-2, der=1)

f = (1+exp((-1)*(w0+w1*x1+w2*x2)))**(-1)

print(f.backprop())


[('w0', 1, 0.14352671126628178), ('w1', 3, 0.4305801337988453), ('x1', 1, 0.14352671126628178), ('w2', 2, 0.28705342253256355), ('x2', -2, -0.28705342253256355)]


In [152]:
print(f.der)

{'w0': 0.19661193324148188, 'w1': 0.5898357997244456, 'w2': 0.39322386648296376, 'x1': 0.19661193324148188, 'x2': -0.39322386648296376}
